In [ ]:
%pip install PyPDF2 -q
%pip install langchain -q
%pip install faiss-cpu -q
%pip install openai -q
%pip install python-dotenv -q
%pip install sentence_transformers -q

In [ ]:
import requests

# Fichero PDF
URL = "Agregar aquí la URL de su PDF"
doc_to_download = requests.get(URL)
# Guardar fichero
pdf_file = open("AdmiteTutela.pdf", "wb")
pdf_file.write(doc_to_download.content)

In [33]:
from PyPDF2 import PdfReader

In [34]:
# Cargar fichero PDF
pdf_file_obj = open('NamePDF.pdf', 'rb')
pdf_reader = PdfReader(pdf_file_obj)

In [35]:
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

In [ ]:
text[0:14000]

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [47]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=100,
    length_function=len
    )

In [48]:
chunks = text_splitter.split_text(text)

In [ ]:
len(chunks)

In [ ]:
chunks[16]

In [56]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' # 471M
'sentence-transformers/paraphrase-multilingual-mpnet-base-v2' #1.11G

In [58]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [59]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
sentence_embeddings = model.encode("agregue aquqí la frase que quiere volver un embeddings")

In [ ]:
len(sentence_embeddings)

In [62]:
from langchain.vectorstores import FAISS

In [63]:
knowledge_base = FAISS.from_texts(chunks, embeddings)

In [67]:
pregunta = "¿Cuales son las pretensiones de la tutela?"
docs = knowledge_base.similarity_search(pregunta, 3)

In [ ]:
docs

In [69]:
# dos pociones

# Primera opción: añadir el api key como variable de entorno
import os
os.environ["OPENAI_API_KEY"] = "Agregue aquí su KEY de OPEN AI"

# Segunda opción: añadir la api key a través de un fichero llamado '.env'
# from dotenv import load_dotenv
# load_dotenv()

In [71]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [72]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo')

chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
pregunta = "¿Haga una pregunta sobre su PDF?"
# Busqueda de párrafos similares
docs = knowledge_base.similarity_search(pregunta, 2)
# Utilizar los parrafos similares para darle contexto a ChatGPT
respuesta = chain.run(input_documents=docs, question=pregunta)
print(f"Respuesta ChatGPT: {respuesta}")

In [75]:
from langchain.callbacks import get_openai_callback

In [ ]:
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=pregunta)
    print(cb)